In [11]:
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [47]:
# 1. 원본 CSV 읽기 (헤더 없이)
df_area = pd.read_csv('dong_area.csv', header=None)

# 2. 3번째 줄을 열 이름으로 설정
# df_area.columns = df_area.iloc[0]

# 3. 위에 있던 0~2행 제거
# df_area = df_area[1:].reset_index(drop=True)

# 4. 전체 열은 유지하고, 일부 열 이름만 변경
#df_area = df_area.rename(columns={
#    '동별(3)': '행정동',
#    '면적 (k㎡)': '면적'
#})

# 5. 결과 저장 (선택 사항)
df_area.to_csv("dong_area_cleaned.csv", index=False)
df_area.head()

,동별(1),동별(2),행정동,면적,구성비 (%),행정,법정,소계,소계
0,서울시,소계,소계,605.20,100.00,426,467,12926,96663
1,서울시,종로구,소계,23.91,3.95,17,87,279,1475
2,서울시,종로구,사직동,1.23,0.20,1,12,16,68
3,서울시,종로구,삼청동,1.49,0.25,1,7,10,42
4,서울시,종로구,부암동,2.27,0.38,1,3,18,98


In [55]:
# 🐶 동물시설 데이터 불러오기
pet_df = pd.read_csv("pet_facilities_data_clean.csv")

# 📏 동 면적 데이터 불러오기 
area = pd.read_csv("dong_area.csv") 

# 🔁 동 이름 좌우 공백 제거 (중요)
area_df['DONG'] = area_df['DONG'].str.strip()
pet_df['DONG'] = pet_df['DONG'].str.strip()

# 🤝 DONG 기준 병합 (left join: pet_df 기준)
merged_df = pd.merge(pet_df, area_df, on='DONG', how='left')

# 💾 저장 또는 확인
print(merged_df[['FCLTY_NM', 'DONG', '면적']].head())
# 또는 저장:
merged_df.to_csv("pet_facilities_data_clean2.csv", index=False)


     FCLTY_NM   DONG    면적
0      100세약국  당산동6가   NaN
1  100코달리 와인바    신당동  0.55
2     1층메디컬약국    시흥동   NaN
3      1층이화약국  하왕십리동   NaN
4    21세기동물병원    보광동  0.71


In [58]:
pet_df = pd.read_csv("pet_facilities_data_clean.csv")
dong_map_df = pd.read_csv("hang_bub_2025.csv")

#hang_bub내용이 pet_facitlities와 맞지 않음 ex) 당산제1동 > 당산 1동이랑 매핑시켜야지 면적을 추가할 수 있음 
dong_map_df['읍면동명'] = dong_map_df['읍면동명'].replace({
    r'제1동$': '1동',
    r'제2동$': '2동',
    r'제3동$': '3동'
}, regex=True)

# 서울특별시만 필터링 및 필요한 열 정리
dong_map_df = dong_map_df[dong_map_df['시도명'] == '서울특별시']
dong_map_df = dong_map_df[['동리명', '읍면동명']].dropna().drop_duplicates()

# 병합 (법정동 → 행정동)
merged_df = pd.merge(pet_df, dong_map_df, left_on='DONG', right_on='동리명', how='left')
merged_df['DONG'] = merged_df['읍면동명']
merged_df.drop(columns=['동리명', '읍면동명'], inplace=True)

# 좌표 기준 중복 제거: LC_LA, LC_LO 조합이 중복된 경우 맨 첫 행만 남김
deduped_df = merged_df.drop_duplicates(subset=['LC_LA', 'LC_LO'], keep='first')
deduped_df.rename(columns={"면적 (k㎡)": "면적km"}, inplace=True)

# 결과 저장 및 출력
deduped_df.to_csv("pet_facilities_with_area2.csv", index=False, encoding='utf-8-sig')

# 확인
print(deduped_df.head())

      FCLTY_NM      LC_LA       LC_LO LNM_ADDR    GU   DONG FCLTY_INFO_DC  \
0       100세약국  37.533260  126.903741    서울특별시  영등포구   당산2동          동물약국   
1   100코달리 와인바  37.563305  127.016417    서울특별시    중구    신당동        반려동물카페   
7      1층메디컬약국  37.453043  126.901712    서울특별시   금천구   시흥1동          동물약국   
12      1층이화약국  37.566590  127.024082    서울특별시   성동구  왕십리2동          동물약국   
14    21세기동물병원  37.528316  126.999196    서울특별시   용산구    보광동          동물병원   

    면적km  
0    NaN  
1   0.55  
7    NaN  
12   NaN  
14  0.71  


/var/folders/vk/04vgyt4s7w58b10wq4bkrvg80000gn/T/ipykernel_83908/1188423943.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deduped_df.rename(columns={"면적 (k㎡)": "면적km"}, inplace=True)


In [63]:
pet_df = pd.read_csv("pet_facilities_with_area2.csv")
area_df = pd.read_csv("dong_area.csv")

# 3. 필요한 열만 추출 및 열 이름 통일
area_df = area_df[['행정동', '면적']].rename(columns={'행정동': 'DONG'})

# 4. 동 이름의 좌우 공백 제거
pet_df['DONG'] = pet_df['DONG'].str.strip()
area_df['DONG'] = area_df['DONG'].str.strip()

# 5. DONG 기준으로 병합 (left join)
merged_df = pd.merge(pet_df, area_df, on='DONG', how='left')

# 6. 결과 저장
merged_df.to_csv("pet_facilities_with_area2.csv", index=False, encoding='utf-8-sig')
remerged = merged_df.drop(columns=['면적km', '면적_y'])

# 7. 매핑 후에도 면적이 NaN인 행의 수 확인
nan_count = remerged['면적'].isna().sum()
print(f"매핑 후에도 면적이 NaN인 행의 수: {nan_count}")


매핑 후에도 면적이 NaN인 행의 수: 28


In [71]:
df = merged_df.drop(['면적km', '면적_x', '면적_y'], axis=1)
df.to_csv("pet_facilities_with_area2.csv", index=False, encoding='utf-8-sig')

In [72]:
df = pd.read_csv("pet_facilities_with_area2.csv")
counts = df["FCLTY_INFO_DC"].value_counts()

dong_counts = df['DONG'].value_counts().reset_index()
dong_counts.columns = ['DONG', '시설수']

# 원본 데이터에 병합
df = df.merge(dong_counts, on='DONG', how='left')

# 결과 출력
print(value_counts)


FCLTY_INFO_DC
동물약국                      1645
반려동물용품                     860
동물병원                       787
반려동물미용                     229
반려동물카페                     128
공원                          23
반려동물카페, 반려동물호텔              19
관광지                         18
반려동물미용, 반려동물호텔              15
반려동물호텔                       7
반려동물미용, 반려동물용품               4
반려동물호텔, 반려동물미용               2
반려동물카페, 반려동물미용, 반려동물호텔       2
반려동물호텔, 반려동물미용, 반려동물용품       1
반려동물카페, 반려동물호텔, 반려동물미용       1
반려동물카페, 반려동물미용               1
반려동물카페,반려동물호텔, 반려동물미용        1
반려동물카페, 반려동물호텔, 반려동물용품       1
반려동물카페, 반려동물용품               1
Name: count, dtype: int64


In [73]:
df

,FCLTY_NM,LC_LA,LC_LO,LNM_ADDR,GU,DONG,FCLTY_INFO_DC,면적,시설수
0,100세약국,37.533260,126.903741,서울특별시,영등포구,당산2동,동물약국,1.55,13.0
1,100코달리 와인바,37.563305,127.016417,서울특별시,중구,신당동,반려동물카페,0.55,24.0
2,1층메디컬약국,37.453043,126.901712,서울특별시,금천구,시흥1동,동물약국,1.71,31.0
3,1층이화약국,37.566590,127.024082,서울특별시,성동구,왕십리2동,동물약국,0.41,7.0
4,21세기동물병원,37.528316,126.999196,서울특별시,용산구,보광동,동물병원,0.71,6.0
...,...,...,...,...,...,...,...,...,...
3740,VIP 동물의료센터,37.592296,127.013413,서울특별시,성북구,동선동,동물병원,0.73,15.0
3741,VIP동물의료센터,37.572010,127.073563,서울특별시,동대문구,장안1동,동물병원,1.25,41.0
3742,VIP동물의료센터 서초동작점,37.479673,126.982192,서울특별시,서초구,서초3동,동물병원,2.93,50.0
3743,W스토아청와약국,37.484566,126.888010,서울특별시,구로구,가리봉동,동물약국,0.40,3.0
